---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

# Assignment 3 - Evaluation

In this assignment you will train several models and evaluate how effectively they predict instances of fraud using data based on [this dataset from Kaggle](https://www.kaggle.com/dalpozz/creditcardfraud).
 
Each row in `fraud_data.csv` corresponds to a credit card transaction. Features include confidential variables `V1` through `V28` as well as `Amount` which is the amount of the transaction. 
 
The target is stored in the `class` column, where a value of 1 corresponds to an instance of fraud and 0 corresponds to an instance of not fraud.

In [ ]:
import numpy as np
import pandas as pd

### Question 1
Import the data from `fraud_data.csv`. What percentage of the observations in the dataset are instances of fraud?

*This function should return a float between 0 and 1.* 

In [ ]:
def answer_one():
    
    # Your code here
    df = pd.read_csv('fraud_data.csv')
    no_instances = df.shape[0]
    fraud = df.loc[df['Class'] == 1]
    fraud_instances = len(fraud.index)
    
    return np.around(fraud_instances / no_instances, decimals=3)


In [ ]:
#answer_one()

In [ ]:
# Use X_train, X_test, y_train, y_test for all of the following questions
from sklearn.model_selection import train_test_split

df = pd.read_csv('fraud_data.csv')

X = df.iloc[:,:-1]
y = df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

### Question 2

Using `X_train`, `X_test`, `y_train`, and `y_test` (as defined above), train a dummy classifier that classifies everything as the majority class of the training data. What is the accuracy of this classifier? What is the recall?

*This function should a return a tuple with two floats, i.e. `(accuracy score, recall score)`.*

In [ ]:
def answer_two():
    from sklearn.dummy import DummyClassifier
    from sklearn.metrics import recall_score
    
    # Your code here
    dummy_majority = DummyClassifier(strategy = 'most_frequent').fit(X_train, y_train) 
    dummy_predicted = dummy_majority.predict(X_test)
    acc = np.around(dummy_majority.score(X_test, y_test),decimals=3)
    rec = np.around(recall_score(y_test, dummy_predicted),decimals=3)
    
    return acc, rec

In [ ]:
#answer_two()

### Question 3

Using X_train, X_test, y_train, y_test (as defined above), train a SVC classifer using the default parameters. What is the accuracy, recall, and precision of this classifier?

*This function should a return a tuple with three floats, i.e. `(accuracy score, recall score, precision score)`.*

In [ ]:
def answer_three():
    from sklearn.metrics import recall_score, precision_score
    from sklearn.svm import SVC

    # Your code here
    cl = SVC().fit(X_train, y_train)
    cl_predicted = cl.predict(X_test)
    
    # get metrics
    acc = cl.score(X_test, y_test)
    rec = recall_score(y_test, cl_predicted)
    prec = precision_score(y_test, cl_predicted)
    
    return acc, rec, prec

In [ ]:
#answer_three()

### Question 4

Using the SVC classifier with parameters `{'C': 1e9, 'gamma': 1e-07}`, what is the confusion matrix when using a threshold of -220 on the decision function. Use X_test and y_test.

*This function should return a confusion matrix, a 2x2 numpy array with 4 integers.*

In [ ]:
def answer_four():
    from sklearn.metrics import confusion_matrix
    from sklearn.svm import SVC

    # Your code here
    cl = SVC(C=1e9, gamma=1e-07).fit(X_train, y_train).decision_function(X_test)    
    scores = [0 if score <= -220 else 1 for score in cl]
    
    return confusion_matrix(y_test, scores)

In [ ]:
#answer_four()

### Question 5

Train a logisitic regression classifier with default parameters using X_train and y_train.

For the logisitic regression classifier, create a precision recall curve and a roc curve using y_test and the probability estimates for X_test (probability it is fraud).

Looking at the precision recall curve, what is the recall when the precision is `0.75`?

Looking at the roc curve, what is the true positive rate when the false positive rate is `0.16`?

*This function should return a tuple with two floats, i.e. `(recall, true positive rate)`.*

In [ ]:
def answer_five():
        
    # Your code here
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import precision_recall_curve, roc_curve, auc

    y_scores_lr = LogisticRegression().fit(X_train, y_train).decision_function(X_test)
    # create precision-recall curve
    precision, recall, thresholds = precision_recall_curve(y_test, y_scores_lr)
    # get the recall value when precision is 0.75
    recall_exact = np.around(recall[np.argmin(abs(precision-0.75))], decimals=4)

    # create roc curve
    fpr_lr, tpr_lr, _ = roc_curve(y_test, y_scores_lr)
    # get the tpr when fpr is close to 0.16
    tpr_exact = np.around(tpr_lr[np.argmin(abs(fpr_lr-0.16))+1],decimals=2)
    print(tpr_exact)
    return recall_exact, tpr_exact

In [ ]:
#answer_five()

### Question 6

Perform a grid search over the parameters listed below for a Logisitic Regression classifier, using recall for scoring and the default 3-fold cross validation.

`'penalty': ['l1', 'l2']`

`'C':[0.01, 0.1, 1, 10, 100]`

From `.cv_results_`, create an array of the mean test scores of each parameter combination. i.e.

|      	| `l1` 	| `l2` 	|
|:----:	|----	|----	|
| **`0.01`** 	|    ?	|   ? 	|
| **`0.1`**  	|    ?	|   ? 	|
| **`1`**    	|    ?	|   ? 	|
| **`10`**   	|    ?	|   ? 	|
| **`100`**   	|    ?	|   ? 	|

<br>

*This function should return a 5 by 2 numpy array with 10 floats.* 

*Note: do not return a DataFrame, just the values denoted by '?' above in a numpy array.*

In [ ]:
def answer_six():    
    from sklearn.model_selection import GridSearchCV
    from sklearn.linear_model import LogisticRegression

    # Your code here
    clf = LogisticRegression()
    grid_values = {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}
    
    # optimisation over grid parameter with recall scoring
    grid_clf_rec = GridSearchCV(clf, param_grid = grid_values, scoring='recall')
    grid_clf_rec.fit(X_train, y_train)
    
    # build a dataframe using split test scores
    df = pd.DataFrame(grid_clf_rec.cv_results_)
    # get the mean for each parameter combination
    output = np.reshape(df['mean_test_score'],(5,2))
    
    return output

In [ ]:
#answer_six()

In [ ]:
# Use the following function to help visualize results from the grid search
def GridSearch_Heatmap(scores):
    %matplotlib notebook
    import seaborn as sns
    import matplotlib.pyplot as plt
    plt.figure()
    sns.heatmap(scores.reshape(5,2), xticklabels=['l1','l2'], yticklabels=[0.01, 0.1, 1, 10, 100])
    plt.yticks(rotation=0);

#GridSearch_Heatmap(answer_six())